# Challenge 2 RAG

Install necessary dependecies

In [37]:
!pip install google-cloud-bigquery google-cloud-aiplatform pandas --quiet

Authenticate GCloud SDK

In [38]:
!gcloud auth application-default login


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=GbHXr6Zp7yq2Usommx67SsnOIe5oSm&prompt=consent&token_

Import necessary packages

In [39]:
from google.cloud import bigquery
from vertexai.preview.language_models import ChatModel
from vertexai.preview.generative_models import GenerativeModel
import vertexai
import pandas as pd

Initialize constants

In [40]:
project_id="qwiklabs-gcp-03-f951e059a60d"
dataset_id="mfarhaan_challenge_2_dataset"
embedding_conn_name="embedding_conn"
embedding_model_name="mfarhaan_challenge_2_embedding_model_id"
embedding_model_id=f"{project_id}.{dataset_id}.{embedding_model_name}"
dataset_table_name="mfarhaan_challenge_2_dataset_table"
dataset_table_id=f"{project_id}.{dataset_id}.{dataset_table_name}"
embedded_dataset_table_name="mfarhaan_challenge_2_dataset_embedded_table"
dataset_embedded_table_id=f"{project_id}.{dataset_id}.{embedded_dataset_table_name}"
vertexai_location="us-central1"
bigquery_location="us"
chatmodel_location="global"
chatmodel_name="gemini-2.0-flash-001"
dataset_src="gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"

Initialize Vertex AI SDK

In [41]:
vertexai.init(project=project_id, location=vertexai_location)

Create a Big Query Client and load the CSV data into big query table

In [42]:
# Big query Load CSV data job configuration
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE"
)

# Initializing Big Query client
bq = bigquery.Client(project=project_id, location=bigquery_location)

#Load CSV data to big query table
load_job = bq.load_table_from_uri(dataset_src, dataset_table_id, job_config=job_config)
load_job.result()
print("CSV Data loaded to Big Query table")

CSV Data loaded to Big Query table


Create an remote embedding model that can be used in future sql queries to vector search the data

In [43]:
#Create an embedding model
embed_model = f"""
  CREATE OR REPLACE MODEL `{dataset_id}.{embedding_model_name}`
  REMOTE WITH CONNECTION `{bigquery_location}.{embedding_conn_name}`
  OPTIONS (ENDPOINT = 'text-embedding-005')
"""
bq.query(embed_model).result()
print("Embedding model created")

Embedding model created


Create a new embedded data table in big query with source table as the csv loaded table. Use remote embedding connection to embed the data

In [44]:
#Create an embedded table with embedded data
embed_table_query = f"""
  CREATE OR REPLACE TABLE `{dataset_id}.{embedded_dataset_table_name}` AS
  SELECT *, ml_generate_embedding_result AS vector_embedding
  FROM ML.GENERATE_EMBEDDING(
    MODEL `{dataset_id}.{embedding_model_name}`,
    (
      SELECT CONCAT(string_field_0, ' ', string_field_1) AS content,
            string_field_0 AS question,
            string_field_1 AS answer
      FROM `{dataset_id}.{dataset_table_name}`
    )
  )
"""
bq.query(embed_table_query).result()
print("Embedded table populated")

Embedded table populated


Create a function to query the data in vector format in big query embedded table return top "K" results as requested

In [45]:
# This function queries the embedded data table. It accepts user's query and retireves top "k" matching results
def query_embdded_table(query, top_k = 3):
    query = f"""
      SELECT query.query, r.base.question, r.base.answer, r.distance
      FROM VECTOR_SEARCH(
        TABLE `{dataset_id}.{embedded_dataset_table_name}`,
        'vector_embedding',
        (
          SELECT
            ml_generate_embedding_result AS vector_embedding,
            '{query}' AS query
          FROM ML.GENERATE_EMBEDDING(
            MODEL `{dataset_id}.{embedding_model_name}`,
            (SELECT '{query}' AS content)
          )
        ),
        top_k => {top_k},
        options => '{{"fraction_lists_to_search": 1.0}}'
      ) AS r
    """
    return bq.query(query).to_dataframe()

Create a chat model to retireve response for user's request

In [46]:
#Create a chat model
chat_model = GenerativeModel(chatmodel_name)

#Create a chat function which accepts user's query
def chat(query):
    print(f"USER: {query}")
    results = query_embdded_table(query)
    context = "\n\n".join([f"Q: {row['question']}\nA: {row['answer']}" for _, row in results.iterrows()])
    prompt = f"""
      Answer to the question based on the context. DO not answer on your own.

      Context:
        {context}

      Question: {query}
    :\n\n{context}\n\nUser: {query}"""
    results = chat_model.generate_content(prompt)
    print(f"SYSTEM: {results.text}")

Intiate RAG questions

In [47]:
chat("When was Aurora bay founded")

USER: When was Aurora bay founded
SYSTEM: Aurora Bay was founded in 1901 by a group of fur traders who recognized the region’s strategic coastal location.

